# Seperability Insight into OPT models
Tests to see if it is possible to remove coding ability from Meta OPT model for different scales.
Current methods are:
- look at activation frequency of MLP mid layers
- Look at 'crossover threshold' of Attention heads

In [ ]:
try: # if in google colab, download necessary python files
  import google.colab 
  ! git clone https://github.com/pesvut/opt-tools.git && mv ./opt-tools/src/*.py .
except ModuleNotFoundError:
  pass
! pip3 install -qqr requirements.txt 

In [ ]:
import torch
import numpy as np
import pandas as pd
import einops
import matplotlib.pyplot as plt
import wandb

from model import Model
from texts import prepare
from activations import prune_and_evaluate, evaluate_all
from data_classes import RunDataHistory

In [ ]:
# Configure initial model and tests
model_size, token_limit  = "1.3b", 1000
run_pre_test             = True
pre_removals = []

# Removals parameters
ff_frac,   ff_eps   = 0.02, 0.001
attn_frac           = 0.01
cripple, focus      = "code", "pile"

In [ ]:
# Prepare data logging
wandb.init(project="seperability-code-pile")
c = wandb.config
c.update({
    "model_size"  : model_size,
    "token_limit" : token_limit,
    "run_pre_test": run_pre_test,
    "ff_frac"  : ff_frac,
    "ff_eps"   : ff_eps,
    "attn_frac": attn_frac,
    "cripple": cripple,
    "focus"  : focus,
})

# Load model and show details about model
history = RunDataHistory()
opt = Model( c.model_size, limit=c.token_limit )

# Pre-pruning of model
opt.delete_ff_keys_from_files(pre_removals)

In [ ]:
# Evaluate model before removal of any neurons
if c.run_pre_test:
    history.add( evaluate_all( opt, 1e5 ) )
    print( history.df.T )

In [ ]:
# First do some pruning of the feed forward layers
for i in range(3):
    data = prune_and_evaluate( opt, c.ff_frac, c.attn_frac, c.ff_eps, cripple=c.cripple, focus=c.focus )
    history.add( data )
print(history.df.T)

In [ ]:
# Next do some pruning of the feed forward layers + attention layers
for i in range(3):
    data = prune_and_evaluate( opt, c.ff_frac, c.attn_frac, c.ff_eps, cripple=c.cripple, focus=c.focus )
    history.add( data )
print(history.df.T)

In [ ]:
print(history.df.T.to_csv())